### Backtesting.py 
### RSI with MA filter
#### https://kernc.github.io/backtesting.py/  

In [36]:
!pip install backtesting
!pip install matplotlib

In [37]:
#to import talib
#from backtesting.test import GOOG
from backtesting import Backtest, Strategy
import yfinance as yf
import pandas as pd
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import talib
import seaborn as sns
import matplotlib.pyplot as plt

In [38]:
# Import symbol
symbols = ["KO"] 
df_multiple_symbols = yf.download(symbols, start="2010-01-01", end="2024-01-01")
# Flatten columns MultiIndex
if isinstance(df_multiple_symbols.columns, pd.MultiIndex):
    df_multiple_symbols.columns = df_multiple_symbols.columns.get_level_values(0)
df_multiple_symbols = df_multiple_symbols.dropna()


[*********************100%***********************]  1 of 1 completed


In [39]:
df_multiple_symbols

Price,Close,High,Low,Open,Volume
Date,,,,,
2010-01-04,17.435404,17.490424,17.392610,17.472084,13870400
2010-01-05,17.224497,17.420127,17.160306,17.377332,23172400
2010-01-06,17.218382,17.252005,17.111397,17.224494,19264600
2010-01-07,17.175583,17.230604,17.041089,17.218378,13234600
2010-01-08,16.857679,17.007457,16.735411,16.952436,28712400
...,...,...,...,...,...
2023-12-22,54.969810,55.101768,54.687045,54.781299,9029400
2023-12-26,55.196022,55.337403,54.724745,54.724745,6422500
2023-12-27,55.337402,55.393957,55.045213,55.271424,8560100


In [40]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import symbol
symbols = ["KO"] 
df_multiple_symbols = yf.download(symbols, start="2010-01-01", end="2024-01-01")
# Flatten columns MultiIndex
if isinstance(df_multiple_symbols.columns, pd.MultiIndex):
    df_multiple_symbols.columns = df_multiple_symbols.columns.get_level_values(0)
df_multiple_symbols = df_multiple_symbols.dropna()

# Using Rsi 2-3 testing with Exponential Moving Average of 233 as the long filter
def optim_func(series):
    # filters out precondition
    if series['Win Rate [%]'] < 75:
        return -1

    return series ['# Trades'] * series ['Return (Ann.) [%]'] 

class RsiOscillator(Strategy):
    
    upper_bound = 80
    lower_bound = 20
    rsi_window = 14
    ema_window = 233
    
    # Rsi input from talib
    def init (self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
        self.ema = self.I(talib.EMA, self.data.Close, self.ema_window)

    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi) and self.data.Close > self.ema:
            self.buy()

bt = Backtest(df_multiple_symbols, RsiOscillator, cash=10000)

# Run optimization
stats, heatmap = bt.optimize(
    upper_bound = range(50, 90, 5),
    lower_bound = range(5, 50, 5),
    rsi_window = range(2, 5),  # Changed from 2 to range(2, 5) for RSI windows 2, 3, 4
    # maximize towards
    maximize = "Sharpe Ratio",
    constraint = lambda param: param.upper_bound > param.lower_bound,
    return_heatmap = True
)

print(heatmap)

# Create heatmap - for each RSI window separately
for rsi_val in [2, 3, 4]:
    print(f"\n{'='*60}")
    print(f"Heatmap for RSI Window = {rsi_val}")
    print(f"{'='*60}")
    
    # Filter heatmap for this RSI window
    heatmap_filtered = heatmap[heatmap.index.get_level_values('rsi_window') == rsi_val]
    
    # Create heatmap for this RSI window
    hm = heatmap_filtered.groupby(["upper_bound", "lower_bound"]).mean().unstack()
    print(f"\nHeatmap Pivot Table (RSI Window = {rsi_val}):")
    print(hm)
    
    # Plot the heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(hm, cmap="plasma", annot=True, fmt=".3f", 
                cbar_kws={'label': 'Sharpe Ratio'})
    plt.title(f'Sharpe Ratio Heatmap: Upper vs Lower RSI Bounds\n(RSI Window = {rsi_val}, EMA = 233)', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Upper Bound')
    plt.ylabel('Lower Bound')
    plt.tight_layout()
    plt.show()

print("\nOptimization Results:")
print(stats)

lower_bound = stats["_strategy"].lower_bound
upper_bound = stats["_strategy"].upper_bound
rsi_window = stats["_strategy"].rsi_window

print(f"\nOptimized Parameters:")
print(f"  RSI Window: {rsi_window}")
print(f"  Lower Bound: {lower_bound}")
print(f"  Upper Bound: {upper_bound}")
print(f"  EMA Window: 233 (fixed)")

# Plot the backtest
bt.plot()

[*********************100%***********************]  1 of 1 completed


In [ ]:
import pandas as pd

# Process the heatmap to get top results
if isinstance(heatmap, pd.Series):
    # Convert Series to DataFrame
    heatmap_df = heatmap.reset_index()
    # The columns will be: upper_bound, lower_bound, rsi_window, and the optim_func value
    print(f"Heatmap columns: {heatmap_df.columns.tolist()}")
    
    # Rename columns properly
    metric_col = heatmap_df.columns[-1]  # Get the name of the metric column
    heatmap_df = heatmap_df.rename(columns={
        heatmap_df.columns[0]: 'upper_bound',
        heatmap_df.columns[1]: 'lower_bound', 
        heatmap_df.columns[2]: 'rsi_window',
        metric_col: 'Optim_Value'
    })
else:
    # It's already a DataFrame
    heatmap_df = heatmap.reset_index()

print(f"\nHeatmap shape: {heatmap_df.shape}")
print(f"Columns: {heatmap_df.columns.tolist()}")

# Filter out invalid results (optim_func returns -1 for failed conditions)
valid_results = heatmap_df[heatmap_df['Optim_Value'] > 0].copy()

if len(valid_results) > 0:
    # Get top 50 by Optim_Value (or all if less than 50)
    top_n = valid_results.nlargest(50, 'Optim_Value').copy()
    
    # Create the results table
    results_list = []
    
    for idx, row in top_n.iterrows():
        # Run backtest with these parameters to get detailed stats
        temp_stats = bt.run(
            upper_bound=int(row['upper_bound']),
            lower_bound=int(row['lower_bound']),
            rsi_window=int(row['rsi_window']),
            ema_window=233  # Fixed as per your strategy
        )
        
        results_list.append({
            'rsi': int(row['rsi_window']),
            'bottom_value': int(row['lower_bound']),
            'upper_value': int(row['upper_bound']),
            'sharpe_ratio': float(temp_stats['Sharpe Ratio']),  # Extracting Sharpe Ratio
            'winrate': float(temp_stats['Win Rate [%]']),
            'total_trades': int(temp_stats['# Trades']),
            'annual_return': float(temp_stats['Return (Ann.) [%]']),
            'max_drawdown': float(temp_stats['Max. Drawdown [%]']),
            'optim_value': float(row['Optim_Value'])  # The value from your optim_func
        })
    
    # Create DataFrame and print
    results_df = pd.DataFrame(results_list)
    
    # Sort by Sharpe Ratio for the display table
    results_df_sorted = results_df.sort_values('sharpe_ratio', ascending=False).reset_index(drop=True)
    
    print("\n" + "="*120)
    print("TOP 50 RESULTS (by Optim Function)")
    print("="*120)
    
    # Print the requested table: Rsi value, lower value, upper value, Sharpe ratio, Winrate
    # Added 'Trades' column to the table
    print(f"{'RSI':^6} {'Bottom':^8} {'Upper':^8} {'Sharpe Ratio':^12} {'Win Rate %':^10} {'Trades':^10}")
    print("-"*70)  # Adjusted for the extra column
    
    for _, row in results_df_sorted.iterrows():
        print(f"{row['rsi']:^6} {row['bottom_value']:^8} {row['upper_value']:^8} {row['sharpe_ratio']:^12.3f} {row['winrate']:^10.2f} {row['total_trades']:^10}")
    
    print("="*120)
    print(f"Total valid results shown: {len(results_df_sorted)} out of {len(valid_results)} valid combinations")
    print("="*120)
    
    # Also print extended table for analysis
    print("\nExtended Results Table (for analysis):")
    print(results_df_sorted[['rsi', 'bottom_value', 'upper_value', 'sharpe_ratio', 'winrate', 
                           'total_trades', 'annual_return', 'max_drawdown', 'optim_value']].to_string(index=False))
    
    # Find the best by Sharpe Ratio
    best_sharpe = results_df.loc[results_df['sharpe_ratio'].idxmax()]
    print(f"\nBest by Sharpe Ratio: RSI={best_sharpe['rsi']}, Bottom={best_sharpe['bottom_value']}, "
          f"Upper={best_sharpe['upper_value']}, Sharpe={best_sharpe['sharpe_ratio']:.3f}, "
          f"Winrate={best_sharpe['winrate']:.2f}%, Trades={best_sharpe['total_trades']}")
    
    # Find the best by Optim Value (your original criteria)
    best_optim = results_df.loc[results_df['optim_value'].idxmax()]
    print(f"Best by Optim Function: RSI={best_optim['rsi']}, Bottom={best_optim['bottom_value']}, "
          f"Upper={best_optim['upper_value']}, Sharpe={best_optim['sharpe_ratio']:.3f}, "
          f"Winrate={best_optim['winrate']:.2f}%, Trades={best_optim['total_trades']}")

Heatmap columns: ['upper_bound', 'lower_bound', 'rsi_window', 'Sharpe Ratio']

Heatmap shape: (216, 4)
Columns: ['upper_bound', 'lower_bound', 'rsi_window', 'Optim_Value']

TOP 50 RESULTS (by Optim Function)
 RSI    Bottom   Upper   Sharpe Ratio Win Rate %   Trades  
----------------------------------------------------------------------
 4.0     10.0     85.0      0.701       100.00      3.0    
 4.0     15.0     85.0      0.614       85.71       7.0    
 4.0     10.0     80.0      0.610       100.00      3.0    
 3.0     5.0      85.0      0.610       100.00      3.0    
 4.0     10.0     75.0      0.540       100.00      3.0    
 2.0     45.0     60.0      0.443       65.43       81.0   
 4.0     5.0      70.0      0.435       100.00      1.0    
 4.0     5.0      75.0      0.435       100.00      1.0    
 4.0     5.0      80.0      0.435       100.00      1.0    
 4.0     5.0      85.0      0.428       100.00      1.0    
 3.0     10.0     85.0      0.422       87.50       8.0    
 